In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

# Define paths
project_dir = '/content/drive/MyDrive/celeb_dataset'
image_folder = os.path.join(project_dir, 'images')
attr_file = os.path.join(project_dir, 'list_attr_celeba.csv')

# Load CSV and fix column name
df = pd.read_csv(attr_file)

# Rename the first column to 'image_id'
df.rename(columns={df.columns[0]: 'image_id'}, inplace=True)

# Normalize -1 to 0
df.replace(-1, 0, inplace=True)

# Filter to only existing image files
existing_images = set(os.listdir(image_folder))
df = df[df['image_id'].isin(existing_images)]

# Limit to 2000 samples
df = df.head(2000).reset_index(drop=True)

# Preview
df.head()


,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,0,1,1,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
1,000002.jpg,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
2,000003.jpg,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
3,000004.jpg,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
4,000005.jpg,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1


In [ ]:
import os

output_folder = "/content/sketches"
os.makedirs(output_folder, exist_ok=True)


In [ ]:
import cv2
from tqdm import tqdm

for filename in tqdm(df['image_id']):
    img_path = os.path.join(image_folder, filename)
    save_path = os.path.join(output_folder, filename)

    # Read image
    img = cv2.imread(img_path)

    if img is None:
        continue

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur (optional, smooths noise)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, threshold1=50, threshold2=150)

    # Save sketch
    cv2.imwrite(save_path, edges)


100%|██████████| 2000/2000 [00:35<00:00, 56.21it/s] 


In [ ]:
# List of attribute columns (excluding the 'image_id')
attribute_cols = df.columns[1:]

# Create a dictionary of {image_id: "attribute text"}
labels = {}

for _, row in df.iterrows():
    active_attrs = [attr for attr in attribute_cols if row[attr] == 1]
    labels[row["image_id"]] = ", ".join(active_attrs)


In [ ]:
import json

with open("/content/labels.json", "w") as f:
    json.dump(labels, f)

# Optional: Save to Drive so you don't lose it
!cp /content/labels.json /content/drive/MyDrive/celeb_dataset/labels.json


In [ ]:
!pip install torch torchvision nltk
!pip install easydict tensorboardX


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
!git clone https://github.com/huggingface/diffusers
%cd diffusers/examples/dreambooth

Cloning into 'diffusers'...
remote: Enumerating objects: 92449, done.
remote: Counting objects: 100% (1731/1731), done.
remote: Compressing objects: 100% (1126/1126), done.
remote: Total 92449 (delta 1229), reused 699 (delta 595), pack-reused 90718 (from 4)
Receiving objects: 100% (92449/92449), 70.61 MiB | 27.56 MiB/s, done.
Resolving deltas: 100% (67704/67704), done.
/content/diffusers/examples/dreambooth


In [ ]:
import json
import pandas as pd

# Load text labels
with open("/content/labels.json", "r") as f:
    labels = json.load(f)

# Build DataFrame
data = [{"image_path": f"/content/sketches/{k}", "prompt": v} for k, v in labels.items()]
df = pd.DataFrame(data)
df.to_csv("/content/celeba_sketch_lora.csv", index=False)

# Preview
df.head()


,image_path,prompt
0,/content/sketches/000001.jpg,"Arched_Eyebrows, Attractive, Brown_Hair, Heavy..."
1,/content/sketches/000002.jpg,"Bags_Under_Eyes, Big_Nose, Brown_Hair, High_Ch..."
2,/content/sketches/000003.jpg,"Big_Lips, Blurry, Male, Narrow_Eyes, No_Beard,..."
3,/content/sketches/000004.jpg,"Attractive, No_Beard, Pointy_Nose, Straight_Ha..."
4,/content/sketches/000005.jpg,"Arched_Eyebrows, Attractive, Big_Lips, Heavy_M..."


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
# Remove the older version completely
!pip uninstall -y diffusers

# Install latest dev version from GitHub (not pip)
!pip install git+https://github.com/huggingface/diffusers.git


Found existing installation: diffusers 0.33.1
Uninstalling diffusers-0.33.1:
  Successfully uninstalled diffusers-0.33.1
  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-7_tr7e2o
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-7_tr7e2o
  Resolved https://github.com/huggingface/diffusers.git to commit 9836f0e000cfd826a7a5099002253ed2becc13e0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.34.0.dev0-py3-none-any.whl size=3687434 sha256=5a0b04d6260dae5d74880773183ee241574180fc57b1b88c877ca743a4781cd2
  Stored in directory: /tmp/pip-ephem-wheel-cache-h36p7w7h/wheels/d2/5c/5f/16639722ea17ecb73ab461b81718584bac08af2801619786b9
Successfully built diffusers


In [ ]:
%cd /content/diffusers/examples/dreambooth

/content/diffusers/examples/dreambooth


In [ ]:
!accelerate launch train_dreambooth_lora.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --instance_data_dir="/content/sketches" \
  --instance_prompt="A sketch of a face" \
  --train_text_encoder \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --max_train_steps=1000 \
  --output_dir="/content/sketch_lora_model"


	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-05-18 17:00:58.612275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747587658.909772    8032 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747587658.988355    8032 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-18 17:00:59.631032: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instruc

In [ ]:
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from peft import PeftModel
import torch
import os

# Load base model
base_model = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    safety_checker=None,
    use_safetensors=True
).to("cuda")

# Load your LoRA weights
pipe.unet.load_attn_procs("/content/sketch_lora_model")

# Use better scheduler for sharper results
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
/usr/local/lib/python3.11/dist-packages/diffusers/loaders/unet.py:212: FutureWarning: `load_attn_procs` is deprecated and will be removed in version 0.40.0. Using the `load_attn_procs()` method has been deprecated and will be removed in a future version. Please use `load_lora_adapter()`.
  deprecate("load_attn_procs", "0.40.0", deprecation_messag

In [ ]:
prompt = "Smiling man with black hair and arched eyebrows in sketch style"
image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]

# Display it
image.show()

# Optional: Save to file
image.save("/content/generated_sketch.png")


  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
prompt = "A young, attractive woman with a pointy nose and straight hair, wearing earrings, lipstick, and a necklace, with no beard, in sketch style."
image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]

# Display it
image.show()

# Optional: Save to file
image.save("/content/generated_sketch2.png")


  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
prompt = "A sketch of a middle-aged man with wavy brown hair and a slight beard. He has bushy eyebrows, narrow eyes, and a straight nose. The man has thick lips and a high cheekbone structure. He is slightly smiling and wearing eyeglasses. The lighting is soft and the sketch is detailed, focusing on facial features."
image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]

# Display it
image.show()

# Optional: Save to file
image.save("/content/generated_sketch3.png")


  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
prompt = "A sketch of a middle-aged woman with wavy short haird. She has bushy eyebrows, narrow eyes, and a straight nose. The woman has thick lips and a high cheekbone structure. She is slightly smiling and wearing eyeglasses. The lighting is soft and the sketch is detailed, focusing on facial features."
image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]

# Display it
image.show()

# Optional: Save to file
image.save("/content/generated_sketch4.png")


  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
# Zip trained LoRA model
!zip -r /content/sketch_lora_model.zip /content/sketch_lora_model

# Zip sketches
!zip -r /content/sketches.zip /content/sketches


  adding: content/sketch_lora_model/ (stored 0%)
  adding: content/sketch_lora_model/pytorch_lora_weights.safetensors (deflated 6%)
  adding: content/sketch_lora_model/logs/ (stored 0%)
  adding: content/sketch_lora_model/logs/dreambooth-lora/ (stored 0%)
  adding: content/sketch_lora_model/logs/dreambooth-lora/1747587707.6151648/ (stored 0%)
  adding: content/sketch_lora_model/logs/dreambooth-lora/1747587707.6151648/events.out.tfevents.1747587707.af6c0c09ac54.8032.1 (deflated 56%)
  adding: content/sketch_lora_model/logs/dreambooth-lora/events.out.tfevents.1747587707.af6c0c09ac54.8032.0 (deflated 69%)
  adding: content/sketch_lora_model/logs/dreambooth-lora/1747587707.6166809/ (stored 0%)
  adding: content/sketch_lora_model/logs/dreambooth-lora/1747587707.6166809/hparams.yml (deflated 53%)
  adding: content/sketch_lora_model/checkpoint-500/ (stored 0%)
  adding: content/sketch_lora_model/checkpoint-500/random_states_0.pkl (deflated 25%)
  adding: content/sketch_lora_model/checkpoint-5

In [ ]:
# Copy zipped model and dataset to your Drive
!cp /content/sketch_lora_model.zip /content/drive/MyDrive/
!cp /content/sketches.zip /content/drive/MyDrive/
!cp /content/labels.json /content/drive/MyDrive/

# Optional: Copy notebook (if you saved it as .ipynb)
# !cp /content/your_notebook_name.ipynb /content/drive/MyDrive/


In [ ]:
!cp /content/Colab Notebooks/nndl.ipynb /content/drive/MyDrive/

cp: cannot stat '/content/Colab': No such file or directory
cp: cannot stat 'Notebooks/nndl.ipynb': No such file or directory


In [ ]:
from google.colab import files
files.download('/content/sketch_lora_model.zip')  # Model
files.download('/content/sketches.zip')         # Dataset
files.download('/content/labels.json')          # Text labels


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
%cd pytorch-CycleGAN-and-pix2pix
!pip install -r requirements.txt


Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2516, done.
remote: Total 2516 (delta 0), reused 0 (delta 0), pack-reused 2516 (from 1)
Receiving objects: 100% (2516/2516), 8.20 MiB | 29.47 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.
/content/pytorch-CycleGAN-and-pix2pix


In [ ]:
import os
import cv2
from tqdm import tqdm

# Paths
sketch_dir = "/content/sketches"
real_dir = "/content/drive/MyDrive/celeb_dataset/images"
output_dir = "/content/datasets/celeba_pix2pix/train"
os.makedirs(output_dir, exist_ok=True)

image_list = sorted(os.listdir(sketch_dir))

for fname in tqdm(image_list):
    sketch_path = os.path.join(sketch_dir, fname)
    real_path = os.path.join(real_dir, fname)

    if not os.path.exists(real_path):
        continue

    # Read and resize both
    sketch = cv2.imread(sketch_path)
    real = cv2.imread(real_path)

    if sketch is None or real is None:
        continue

    sketch = cv2.resize(sketch, (256, 256))
    real = cv2.resize(real, (256, 256))

    # Concatenate: [sketch | real]
    combined = cv2.hconcat([sketch, real])

    # Save to output
    cv2.imwrite(os.path.join(output_dir, fname), combined)


100%|██████████| 2000/2000 [00:12<00:00, 156.41it/s]


In [ ]:
!mkdir -p /content/datasets/celeba_pix2pix/test


In [ ]:
%cd /content/pytorch-CycleGAN-and-pix2pix


/content/pytorch-CycleGAN-and-pix2pix


In [ ]:
%cd /content/pytorch-CycleGAN-and-pix2pix

!python train.py --dataroot /content/datasets/celeba_pix2pix \
  --name celebA_pix2pix \
  --model pix2pix \
  --direction AtoB \
  --batch_size 1 \
  --n_epochs 100 \
  --n_epochs_decay 0 \
  --save_epoch_freq 10 \
  --display_id -1


/content/pytorch-CycleGAN-and-pix2pix
----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: /content/datasets/celeba_pix2pix	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            	[default: 1]
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                   

In [ ]:
!rm -rf /content/pytorch-CycleGAN-and-pix2pix/checkpoints/celebA_pix2pix


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/sketch_lora_model.zip -d /content/sketch_lora_model


In [ ]:
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float32,         # Use float32 instead of float16 on CPU
    safety_checker=None,
    use_safetensors=True
)

# Load LoRA weights from your folder
pipe.unet.load_attn_procs("/content/sketch_lora_model")


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


OSError: Error no file named pytorch_lora_weights.bin found in directory /content/sketch_lora_model.